# In Class Project






## Problem formulation

The aim of this final lab is to give you the possibility to work on a sample project. This will give you a better grasp of how the final project will be conducted and allow you more coding practice compared to previous labs.

The sample project is inspired by the paper [Mind the Gap: Understanding the Modality Gap in Multi-modal Contrastive Representation Learning](https://arxiv.org/pdf/2203.02053.pdf). During the course, we discussed how Vision-Language Models (VLMs) can align representations from language and vision. This alignment can be achieved using contrastive learning on image/captions pairs. However, the paper shows that performing a dimensionality reduction technique on embedded image/caption pairs results in the two modalities being disjointed in the embedding space. You can better visualize this through the following image:

<img src="https://modalitygap.readthedocs.io/en/latest/_images/Figure1.png" width="600">

As shown in the image, the "gap" between the two modalities is present for a randomly initialized network and persists even after the pretraining phase. Moreover, this modality gap is not only present in images/text pairs but also when text is aligned with other modalities (videos, medical images, amino-acid sequences).

Geometrically, the authors talk about a "cone effect," which means that with a growing number of dimensions, embeddings tend to occupy smaller regions of the space assuming a cone-like shape.

Your task will consist of testing whether the "modality gap" and "cone effect" exist using video/caption pairs and a state-of-the-art VLM model, [COCA](https://arxiv.org/pdf/2205.01917.pdf).

## Overview of the project

The project is divided into three main phases, explained in detail in the following sections. These phases are sequential, and you can perform all of them here on Colab using the available T4 GPU.

### Step 1: Video Captioning

Your first task is to perform Video Captioning, which is the automatic captioning of a video by understanding the actions and events in it.

We will provide you with short videos and no captions. Then, you'll have to extract a number of frames from each video and generate an independent caption for each frame using COCA.

You are free to choose how many frames to extract from each video, how many of these frames will be used to generate captions, how many captions to retain for each video, and which strategy to use to generate captions.

As you can imagine, many frames of a single video might be repetitive and lead to the same caption, and conversely, few key frames might contain different actions that are necessary to understand the dynamic of the event represented in the video (For example, think about a tennis player serving; probably the initial frames of the video will depict the player in a static position, focusing and preparing to serve, whereas the act of serving will be present only in fewer frames). There are multiple solutions to this problem, and you are free to choose one. Some examples are:

* Subsample only a smaller number of frames and generate captions only for those (faster).
* Filter captions based on their diversity. An approach is to cluster similar captions using the text encoder of COCA and take only one caption for each cluster (slower).

### Step 2: Caption Aggregation

At the end of step 1, you'll have a collection of captions for each single video, with these captions describing only some frames but not the video overall.

Your second task is to obtain a single summary describing the content of a video by aggregating the content of your captions. To this end, you have to choose a LLM and prompt it to generate an overall description of a video given a list of captions.

As an LLM you can use a model from the [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) family. These models have good instruction-following capabilities and work well out-of-the-box. If you have any other preference, you are free to choose other LLMs.

P.S. After doing some experiments I found the [Phi-1.5](https://huggingface.co/microsoft/phi-1_5) model to be a good compromise in terms of memory requirements and performance. This model had no training to follow instructions, so to make it do what you want you will have to provide in-context examples and see how it performs in a few shots manner. See image below:

<img src="https://ai.stanford.edu/blog/assets/img/posts/2022-08-01-understanding-incontext/images/image11.gif" width="600">


### Step 3: Dimensionality Reduction

Once you have paired videos and captions, you are ready to see if the "modality gap" is present for these data points.

Your task is now to encode both videos and captions and use U-MAP dimensionality reduction to project the embeddings in a 2-D space. Since videos are composed of multiple frames, you'll have to use a fusion strategy to aggregate the embedding of all frames. For example, you can take the average of the embeddings to represent a whole video.

If the results are as expected, at this point, you should see two different clusters of data points, each representing one modality.

## Getting started

All the tools necessary to perform these tasks were provided during the course of previous labs.

You can also find here a list of tools.

* **FFMPEG TO EXTRACT FRAMES FROM VIDEOS**

  This is the command for extracting frames from a video specifying the frame rate using FFmpeg:

  ```
  ffmpeg -i input.mp4 -vf fps=1 %04d.png
  ```

  `%04d.png` is a sequence pattern type used to interpret the output file names by sequencing them with zero-padded sequential numbers, eg. 0001.png, 0002.png, 0003.png, etc.

* **OPEN CLIP TO ACCESS THE COCA MODEL**

  This is the [link](https://github.com/mlfoundations/open_clip) to the Open CLIP repository.

* **HUGGINGFACE'S TRANSFORMERS TO USE FLAN-T5 OR PHI-1.5**

  This is the [link](https://huggingface.co/docs/transformers/model_doc/flan-t5) to the FLAN-T5 models documentation on HuggingFace. This is the [link](https://huggingface.co/microsoft/phi-1_5) to Phi-1.5.

* **U-MAP FOR DIMENSIONALITY REDUCTION**

  This is the official [documentation](https://umap-learn.readthedocs.io/en/latest/basic_usage.html) of the python implementation of U-MAP.




# Step 1.0: Get the data

In [ ]:
from datasets import load_dataset
from huggingface_hub import snapshot_download

### Get the video-caption pairs

In [ ]:
# Load dataset (videos with captions)
dataset = load_dataset("friedrichor/ActivityNet_Captions")

# Keep only a sample of 1000 examples for testing
df = dataset['train'].take(10)

# Visualize first sample
print(f"Samples: {len(df)} || ITEM 0:", df[0])

### Get raw videos

In [ ]:
from os import path, rename, listdir, remove

In [ ]:
video_folder = path.join('data', 'videos')
if not path.exists(video_folder):
    
    # Download the files from Hugging Face
    snapshot_download(repo_id="friedrichor/ActivityNet_Captions", repo_type="dataset", allow_patterns=["*.tar.part-00*"], local_dir='./data/raw') 
    
    # Combine the parts into a single tar file and extract it
    !cat ./data/raw/ActivityNet_Videos.tar.part-* | tar -vxf - -C ./data
    
    # Rename the extracted folder to a simpler name
    rename(path.join('data', 'Activity_Videos'), video_folder)
    
    # Delete the parts to save space
    !rm -r -f ./data/raw
    
    # Keep only the sampled videos 
    sampled_video_ids = set(item['video_id'] for item in df)
    for video_file in listdir(video_folder):
        video_id = video_file.split('.')[0]
        if video_id not in sampled_video_ids:
            remove(path.join(video_folder, video_file))
            
# Print the number of videos downloaded
print('Raw videos:', len(listdir(video_folder)))

# Step 1.1: Extract frames from videos

In [ ]:
!apt install ffmpeg

In [ ]:
import os
import re
#import gdown
import subprocess
from glob import glob
from os import makedirs

In [ ]:
# define a directory to store video frames and create it
video_frames_dir = path.join("data", "frames")
if not os.path.exists(video_frames_dir):
    os.makedirs(video_frames_dir)

# iterate through videos
for i, video in enumerate(glob("data/videos/*")):
    video_name = os.path.basename(video).split(".")[0]

    # create new directory to store frames from a video
    frames_dir = path.join(video_frames_dir, video_name)
    makedirs(frames_dir, exist_ok=True)
        
    # define ffmpeg command
    ffmpeg_command = [
        'ffmpeg/bin/ffmpeg',
        '-i', video,
        '-vf', 'fps=1',
        f'{frames_dir}/%02d.png'
    ]
    
    # run command
    subprocess.call(ffmpeg_command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Step 1.2: Caption video frames

In [ ]:
!pip install -q open_clip_torch

In [ ]:
from collections import defaultdict
from PIL import Image
import open_clip
import torch
from tqdm import tqdm

### OpenCLIP, an open source implementation of OpenAI's CLIP (Contrastive Language-Image Pre-training)

In [ ]:
device = torch.device('cuda')

# Instantiate CLIP model
model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k",
  device=device
)

In [ ]:
# set model to evaluation mode
model.eval()

# iterate through video directories
captions_dict = defaultdict(list)
for video_dir in tqdm(glob(f"{video_frames_dir}/*")):
    
    # get video name
    video_id = os.path.basename(video_dir)
    
    # iterate through frames in the video directory
    frame_files = sorted(glob(f"{video_dir}/*"))
    for frame_path in frame_files:
        
        # preprocess frame
        im = Image.open(frame_path).convert("RGB")
        im = preprocess(im).unsqueeze(0).to(device)

        # generate caption for frame
        with torch.no_grad():
            generated = model.generate(im)

        # add generated caption to dictionary
        caption = open_clip.decode(generated[0]).replace("<start_of_text>", "").replace(" <end_of_text>", "").strip()
        captions_dict[video_id].append(caption)

# Clean up memory
del model
torch.cuda.empty_cache()

### Display captions for first video

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Display captions for first video
first_video_id = list(captions_dict.keys())[0]

# Select a number of frames to visualize
num_frames_to_viz = 6
generated_captions = captions_dict[first_video_id][:num_frames_to_viz]

# Create the main figure
ncols = 2
nrows = (num_frames_to_viz + 1) // ncols

fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(6 * ncols, 3 * nrows))

fig.suptitle(f"Captions for video '{first_video_id}'", fontsize=16, color = 'firebrick')

# Iterate through frames and captions and create subplots
for idk, ax in enumerate(axes.flatten()):
    
    # Get the caption
    caption = generated_captions[idk]
    
    # Load the image from the disk
    img = Image.open(path.join(video_frames_dir, first_video_id, f"{idk+1:02d}.png"))
    
    # Visualize the image on the notebook
    ax.imshow(img)
    ax.set_title(f'[FRAME {idk + 1}]\nGENERATED: "{caption}')
    ax.axis('off')

# Visualize the figure
fig.tight_layout()
plt.show()

# Step 2: Use a LLM to generate a single caption for a video 
TASK: Aggregate and summarize the frame captions

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = 'microsoft/Phi-4-mini-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
# Set model to evaluation mode
model.eval()

# Iterate through videos
aggregated_captions = dict()
for video_id in tqdm(captions_dict):
    
    # Aggregate all the frame captions to generate a context
    video_sentences = '\n'.join(captions_dict[video_id])

    # Define the instruction prompt with (1) task description, (2) in-context examples, and (3) input sentence.
    prompt = f"""
    Instruction:
    Create a summary sentence that aggregate the meaning of all the sentences provided in the context. The sentences in the context are in cronological order. Provide a concise summary using only the information provided in the context.

    Context:
    The two kids are playing with the cat
    The cat is runnning in the living room .
    The cat is runnning in the living room .
    Two kids are trying to catch a black cat
    One child is running in the living room .
    The cat is runnning in the living room .
    A woman is holding a black cat .
    The children and a woman are petting the cat .

    Summary:
    Two children run into the living room trying to catch a black cat. After a woman catches the cat, they all pet it together .

    Context:
    {video_sentences}

    Summary:
    """
    
    # Generate aggregate captions using the LLM
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, repetition_penalty=1.2, max_new_tokens=50)
    generated_text = tokenizer.decode(outputs[0][len(inputs.input_ids.squeeze()):], skip_special_tokens=True).strip()
    
    # Attach the summarized caption to the video
    aggregated_captions[video_id] = generated_text

#### Visualize the aggregated caption

In [ ]:
video_id = list(aggregated_captions.keys())[0]

print("AGGREGATED:", aggregated_captions[video_id])
print(f"ORIGINAL CAPTIONS ({len(glob(path.join('data', 'frames',video_id, '*.png')))} frames || {len(captions_dict[video_id])} captions):")
for frame_id, caption in enumerate(captions_dict[video_id]):
    print(f'- FRAME {frame_id + 1}: "{caption}"')

# Step 3: Project video/caption emebeddings in a low dimensional space

In [ ]:
!pip install -q umap-learn

In [ ]:
import numpy as np
from open_clip.factory import get_tokenizer
from umap import UMAP
import matplotlib.pyplot as plt

In [ ]:
# instantiate again COCA
model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k",
  device=device
)

In [ ]:
video_embeddings = []

for video_id in aggregated_captions_dict:
    video_features = []
    frame_files = sorted(glob(f"{video_frames_dir}/{video_id}/*"))
    for frame_path in frame_files:
        im = Image.open(frame_path).convert("RGB")
        im = preprocess(im).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(im).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        video_features.append(image_features.cpu().numpy())
    
    # a video embedding is the average of the frames embeddings
    if video_features:  # check if we have frames
        video_embeddings.append(np.mean(np.asarray(video_features), axis=0))

video_embeddings = np.asarray(video_embeddings).squeeze()
print(video_embeddings.shape)

In [ ]:
tokenizer = get_tokenizer("coca_ViT-L-14")

caption_embeddings = []

for video_id in aggregated_captions_dict:
    text_tokens = tokenizer(aggregated_captions_dict[video_id]).to(device)
    # the following line fix a bug - handle case where no padding tokens exist
    if torch.any(text_tokens == 0):
        text_tokens = text_tokens[:, :torch.where(text_tokens == 0)[1][0]]
    with torch.no_grad():
        text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    caption_embeddings.append(text_features.cpu().numpy())

caption_embeddings = np.asarray(caption_embeddings).squeeze()
print(caption_embeddings.shape)

In [ ]:
# instantiate umap
reducer = UMAP()

# obtain 2d features
features_2d = reducer.fit_transform(np.concatenate([video_embeddings, caption_embeddings], 0))

# plot 2d features
plt.scatter(features_2d[:-len(video_embeddings), 0], features_2d[:-len(video_embeddings), 1], c='tab:blue', label="video")
plt.scatter(features_2d[-len(video_embeddings):, 0], features_2d[-len(video_embeddings):, 1], c='tab:red', label="text")
# plot lines
for i in range(len(video_embeddings)):
    plt.plot([features_2d[i, 0], features_2d[len(video_embeddings)+i, 0]], [features_2d[i, 1], features_2d[len(video_embeddings)+i, 1]], c='black', alpha=0.1)

plt.xlabel('umap 1')
plt.ylabel('umap 2')
plt.legend()
plt.show()

In [ ]:
# define svd
def svd(X, n_components=2):
    U, S, Vt = np.linalg.svd(X)
    return U[:, :n_components] * S[:n_components]

# obtain 2d features
features_2d = svd(np.concatenate([video_embeddings, caption_embeddings], 0))

# plot 2d features
plt.scatter(features_2d[:-len(video_embeddings), 0], features_2d[:-len(video_embeddings), 1], c='tab:blue', label="video")
plt.scatter(features_2d[-len(video_embeddings):, 0], features_2d[-len(video_embeddings):, 1], c='tab:red', label="text")
# plot lines
for i in range(len(video_embeddings)):
    plt.plot([features_2d[i, 0], features_2d[len(video_embeddings)+i, 0]], [features_2d[i, 1], features_2d[len(video_embeddings)+i, 1]], c='black', alpha=0.1)

plt.xlabel('svd 1')
plt.ylabel('svd 2')
plt.legend()
plt.show()